# A.9 성능 팁

- NumPy를 활용하는 코드에서 좋은 성능을 이끌어내는 방법은 꽤 직관적인데, 순수 파이썬 반복문은 상대적으로 매우 느리므로 일반적으로 배열 연산으로 대체한다.
- 다음은 염두에 두면 좋은 간략한 팁이다.
    - 파이썬 반복문과 조건문을 배열 연산과 불리언 배열 연산으로 변환한다.
    - 가능하면 브로드캐스팅을 사용한다.
    - 배열의 뷰(슬라이스)를 사용해서 데이터를 복사하는 것을 피한다.
    - ufunc 메서드를 활용한다.

- NumPy만으로 원하는 성능을 이끌어내지 못한다면 코드를 C나 포트란으로 작성하거나 아니면 Cython을 사용해서 성능을 높일 수 있다.
- Cython의 경우는 최소한의 개발 노력으로 쉽게 C 수준의 성능을 이끌어낼 수 있다.

# A.9.1 인접 메모리의 중요성

- 어떤 애플리케이션에서는 배열이 메모리상에 배치된 모양에 따라 연산 속도에 많은 영향을 끼친다.
- 이는 부분적으로 CPU의 캐시 구조에 의한 성능 차이에 기반하는데 연속된 메모리에 접근하는 연산(예를 들어 C 순서로 저장된 배열에서 로우를 합산하는)의 경우,
  메모리 서브시스템이 적절한 메모리 블록을 매우 빠른 CPU의 L1이나 L2에 저장하게 되므로 가장 빠르다.
- 또한 NumPy의 C 코드 기반 내부의 어떤 코드는 연속된 메모리일 경우 최적화되어 인접하지 않은 메모리를 읽는 문제를 회피할 수 있다.

- 배열이 메모리상에 연속적으로 존재한다는 의미는 배열의 원소가 실제 배열 상에서 나타나는 모습대로(포트란의 컬럼 우선 순서, C의 로우 우선 순서) 메모리에 저장되었다는 의미다. 기본적으로 NumPy 배열은 메모리에 C 순서 혹은 그냥 단순히 연속적으로 생성된다.
- C 순서로 저장된 배열의 전치 배열 같은 컬럼 우선 순서 배열은 포트란 순서 배열이라고 할 수 있다.
- 이 속성을 ndarray의 flags 속성을 통해 명시적으로 확인할 수 있다.

In [1]:
import numpy as np
arr_c = np.ones((1000, 1000), order="C")

In [2]:
arr_f = np.ones((1000, 1000), order="F")

In [3]:
arr_c.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [4]:
arr_f.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [5]:
arr_f.flags.f_contiguous

True

- 이 예제에서 배열의 로우의 합은 메모리에서 로우가 연속적으로 존재하므로 이론적으로 arr_c가 arr_f보다 빠르게 계산된다.
- %timeit을 사용해서 성능치를 확인해볼 수 있다.

In [6]:
%timeit arr_c.sum(1)

768 µs ± 67.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
%timeit arr_f.sum(1)

478 µs ± 116 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


- 이는 NumPy에서 성능을 더 짜내야 할 때 더 많이 노력을 기울이게 되는 부분이다.
- 원하는 메모리 순서로 저장되지 않은 배열이 있다면 그 배열을 "C"나 "F" 순서로 복사해서 사용할 수 있다.

In [8]:
arr_f.copy("C").flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

- 한 배열에 대한 뷰를 생성할 때 그 결과가 항상 연속된 메모리에 할당되지 않을 수 있다는 점을 기억하자.

In [9]:
arr_c[:50].flags.contiguous

True

In [10]:
arr_c[:, :50].flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False